In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import root_mean_squared_error


df_original = pd.read_csv("data/diabetes.csv")

df = df_original.copy()
df.rename(columns={"DiabetesPedigreeFunction":"PedigreeFunc."},inplace=True)

fig, ax = plt.subplots(2,2, figsize=(14,12))

sns.boxplot(df,y="Glucose",hue="Outcome",ax=ax[0,0])
ax[0,0].set_title("Distribución de 'Glucose' en 'Outcome'")
ax[0,0].grid("on")

sns.barplot(df,x="Outcome",y="Glucose",ax=ax[0,1])
ax[0,1].set_title("Media de 'Glucose' en 'Outcome'")

sns.heatmap(df.corr(),annot=True,ax=ax[1,0])
ax[1,0].set_title("Mapa de calor para cada correlación pearson")

sns.scatterplot(df,x="Glucose",y="Insulin",hue="Outcome",ax=ax[1,1])
ax[1,1].set_title("Correlación entre 'Glucose' y 'Insulin'")
ax[1,1].grid("on")

plt.show()

#### Feature Engireering
Es el proceso de seleccionar las variables indicadas, eliminar las que no sirven y transformar las que requieren un preprocesamiento para potenciar la eficazia de un modelo. Una técnica implementada es el Análisis de Componentes Principales(PCA), cuyo fundamento se basa en conceptos de álgebra lineal y su objetivo en la reducción de la dimensionalidad pero conservando a su vez la mayor cantidad de información, proyectando una especie de sombra en los datos. Se implementa a la hora de combatir la maldición de la dimensionalidad, permitir crear gráficos cartesianos y facilitar el trabajo a algoritmos que no sean robustos, como en este ejemplo.

In [ ]:
# seleccionando las variables más explicativas
df = df.loc[:,["Glucose","Insulin","BMI","SkinThickness","Outcome"]]

pca_model = PCA(n_components=2) 

pca = pca_model.fit_transform(df[df.columns[:-1]])

# reduciendo esas variables a una dimensión de 2 componentes
df["PCA_1"] = pca.T[0]
df["PCA_2"] = pca.T[1]

df = df.loc[(df["PCA_1"] < 550) & (df["PCA_2"] < 100),:]

# Crear un gráfico de dispersión PCA 
plt.figure(figsize=(11,4))
plt.scatter(df.loc[df["Outcome"] == 0, "PCA_1"], df.loc[df["Outcome"] == 0, "PCA_2"], label="Clase 0")
plt.scatter(df.loc[df['Outcome'] == 1, 'PCA_1'], df.loc[df['Outcome'] == 1, "PCA_2"], label="Clase 1")
plt.grid("on")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Figura PCA")
plt.legend()
plt.show()

#### Sobreajuste
En la siguiente celda se extrajeron específicamente un par de variables continuas formando un subconjunto sin contexto y que no brinda ninguna información en particular, esto se hizo para explicar un concepto recurrente en el aprendizaje automático y que está fuertemente relacionado con otra definición vista en el primer proyecto(análisis y predicción de rotación de empleados), siendo esta el error de varianza. El sobreajuste consiste en un modelo que se que se adaptó de forma elevada un grupo de datos, dejándolo poco preparado para afrontar nuevas observaciones. En la anterior figura se muestran las diferentes selecciones de datos de entrenamiento y prueba de cada modelo junto con su función objetivo que varía según las muestras. En este caso, se encuentra un umbral que diferencia dos agrupaciones con el fin de ilustrar un modelo conservador a sus objetos y como el cambio de elección de datos provoca un nuevo desempeño de este.


In [ ]:
iris = sns.load_dataset("iris")

iris["species_encoded"] = LabelEncoder().fit_transform(iris["species"])
pca_iris = pca_model.fit_transform(iris[iris.columns[:4]])

iris["PCA_1"] = pca_iris.T[0]
iris["PCA_2"] = pca_iris.T[1]

var = iris.loc[(iris["species"] == "virginica") | (iris["species"] == "setosa"), ["PCA_1","PCA_2"]]
var["PCA_1"] = round(var["PCA_1"]*100).astype(int)
var["PCA_2"] = round(var["PCA_2"]*100).astype(int)

def add_values(length, value_min, value_max):
    new_values = np.array([])
    for i in range(round(length)):
        value = random.randint(value_min, value_max)
        new_values = np.append(new_values, value)
    return new_values
    
grp1_pca_1 = add_values(len(var)/2, 100, 400)
grp1_pca_2 = add_values(len(var)/2, -100, 100)
grp2_pca_1 = add_values(len(var)*1.5, -400, -200)
grp2_pca_2 = add_values(len(var)*1.5, -100, 100)

new_data_1 = np.array([grp1_pca_1,grp1_pca_2]).T
new_data_2 = np.array([grp2_pca_1,grp2_pca_2]).T

df_new_data_1 = pd.DataFrame(new_data_1, columns=["PCA_1","PCA_2"])
df_new_data_2 = pd.DataFrame(new_data_2, columns=["PCA_1","PCA_2"])

var = pd.concat([var, df_new_data_1, df_new_data_2])

fig, ax = plt.subplots(1,3, figsize=(18,7))

def get_function_objetive(i, data_split):
    var["data_split"] = data_split
    knn_regressor = KNeighborsRegressor(n_neighbors=150)
    knn_regressor.fit(var.loc[var["data_split"] == "data_train", "PCA_2"].values.reshape(-1,1), var.loc[var["data_split"] == "data_train", "PCA_1"])
    RMSE = root_mean_squared_error(knn_regressor.predict(var.loc[var["data_split"] == "data_test", "PCA_2"].values.reshape(-1,1)), var.loc[var["data_split"] == "data_test", "PCA_1"])
    sns.scatterplot(var, x="PCA_2", y="PCA_1", hue="data_split", ax=ax[i])
    ax[i].grid("on")
    ax[i].set_title(f"RMSE: {str(RMSE)[:5]}")

get_function_objetive(0, np.where(var["PCA_1"] < -195, "data_train", "data_test"))
get_function_objetive(1, np.where((var["PCA_1"] <= -200) & (var["PCA_2"] < 40) | (var["PCA_1"] > 0) & (var["PCA_2"] > 0), "data_train", "data_test"))
get_function_objetive(2, np.where((var["PCA_1"].between(-300,-200)) | (var["PCA_1"].between(200,400)), "data_train", "data_test"))

plt.show()

#### Bagging 
es una de las principales técnicas computacionales de estadística inferencial cuya objetivo se basa en eliminar o reducir lo máximo posible la varianza en los resultados, su método es utilizar la sabiduría de las masas, es decir, reforzar el aprendizaje mediante el criterio de distintos metaestimadores que se encargarán de sacar sus propias conclusiones de respectivos datos seleccionados para luego, dependiendo del problema en cuestión, llegar a una respuesta final a través de la media o moda del conjunto de resultados.

In [ ]:
# ajustando modelo a través de PCA y KNN
KNN_classifier = KNeighborsClassifier(n_neighbors=5)

bagging_model = BaggingClassifier(estimator=KNN_classifier, # clasificador base
                        n_estimators=100, # cantidad de estimadores
                        max_samples=0.3) # número de muestras requeridas para cada modelo
bagging_model.fit(df[["PCA_1","PCA_2"]].values, df["Outcome"])

object = [[df["Glucose"].mean(), df["Insulin"].mean(), df["BMI"].mean(), df["SkinThickness"].mean()]]

pca_object = pca_model.transform(object)

predict_encoded = bagging_model.predict(pca_object)

map_diabetes = {
    1:"LP",
    0:"HP"
}

df["Outcome_no_encoded"] = df["Outcome"].apply(lambda x : map_diabetes.get(x))

classes = list(zip(df["Outcome_no_encoded"].unique(),df["Outcome"].sort_values().unique()))

# asociando la predicción a su clase

predict_example = classes[predict_encoded[0]][0]

probability = bagging_model.predict_proba(pca_object)

probability = probability[0, predict_encoded]*100

probability = str(probability[0])

print("-------------------------------------")
print("Nuevo objeto \n")
for c in df.columns[:4]:
    print(f"{c}: {round(df[c].mean(),2)}")
    
print(f"\npredicción: {predict_example} | probabilidad: {probability[:4]}%")
print("-------------------------------------")

#### Dashboard interactivo que informa sobre medias y permite introducir datos para clasificarlos


In [ ]:
app = dash.Dash(__name__)

high_progression = df.loc[df["Outcome"] == 1,:]
low_progression = df.loc[df["Outcome"] == 0,:]

colors_outcome = {
    "LP":"green",
    "HP":"red",
}

predict_text = html.B(children=[],id="predict",style={})
probability_text =  html.B(children=[],id="probability")

graph_pca = go.Figure()
graph_pca.add_trace(go.Scatter(x=high_progression["PCA_1"],y=high_progression["PCA_2"],mode="markers",marker_color="red",name="HP"))
graph_pca.add_trace(go.Scatter(x=low_progression["PCA_1"],y=low_progression["PCA_2"],mode="markers",marker_color="green",name="LP"))
graph_pca.update_layout(title="PCA(principal components analysis)")
graph_pca.update_layout(legend=dict(font=dict(size=9)))

app.layout =  html.Div(id="body",className="e4_body",children=[
    html.H1("Diabetes's Progression",id="title",className="e4_title"),
    html.Div(id="dashboard",className="e4_dashboard",children=[
        html.Div(className="e4_info_div",children=[
           html.H2("Diabetes's Means",id="title_2",className="e4_title_2"),
           html.Div(className="e4_info",children=[
              html.Img(id="img",className="e4_img", src="assets/diabetes.jpg"),
              html.Div(className="e4_ul_div",children=[
              html.Ul(className="e4_ul", children=[
                html.H3("High progression",className="e4_title_2"),
                html.Li(f"Glucose: {round(high_progression["Glucose"].mean(),1)}"),
                html.Li(f"Insulin: {round(high_progression["Insulin"].mean(),1)}"),
                html.Li(f"BMI: {round(high_progression["BMI"].mean(),1)}"),
                html.Li(f"Skin Thickness: {round(high_progression["SkinThickness"].mean(),1)}")
              ]),
              html.Ul(className="e4_ul", children=[
                html.H3("Low progression",className="e4_title_2"),
                html.Li(f"Glucose:{round(low_progression["Glucose"].mean(),1)}"),
                html.Li(f"Insulin: {round(low_progression["Insulin"].mean(),1)}"),
                html.Li(f"BMI: {round(low_progression["BMI"].mean(),1)}"),
                html.Li(f"Skin Thickness:{round(low_progression["SkinThickness"].mean(),1)} ")
              ])
            ])
           ])
        ]),
        html.Div(className="e4_graph_div",children=[
            dcc.Graph(id="graph-2",className="e4_graph",figure=graph_pca),
            html.Form(id="input_div",className="input_div",children=[
                dcc.Input(id="input_1",className="input",type="text",placeholder="Glucose",size="7"),
                dcc.Input(id="input_2",className="input",type="text",placeholder="Insulin",size="7"),
                dcc.Input(id="input_3",className="input",type="text",placeholder="BMI",size="7"),
                dcc.Input(id="input_4",className="input",type="text",placeholder="Skin thickness",size="7"),
                html.Button("enviar",id="button",type="button",className="input",n_clicks=0)
            ]),
            html.P(["pred.: ",predict_text," | prob.: ",probability_text,"%"],className="e4_predict")
        ])
    ])
])
        
@app.callback(
    [Output(component_id="graph-2",component_property="figure"),
    Output(component_id="predict",component_property="children"),
    Output(component_id="probability",component_property="children"),
    Output(component_id="predict",component_property="style"),
    Output(component_id="probability",component_property="style")],
    [Input(component_id="button",component_property="n_clicks")],
    [State(component_id="input_1",component_property="value"),
    State(component_id="input_2",component_property="value"),
    State(component_id="input_3",component_property="value"),
    State(component_id="input_4",component_property="value")]
)

def update_graph(n_clicks, var_1, var_2, var_3, var_4):
    if n_clicks is not None and n_clicks > 0:
        object = [[var_1,var_2,var_3,var_4]]    
        pca_object = pca_model.transform(object)
        predict_encoded = bagging_model.predict(pca_object)
        predict = classes[predict_encoded[0]][0]
        predict_color = {"color":colors_outcome[predict]}
        probability_color = {"color":colors_outcome[predict]}
        probability = bagging_model.predict_proba(pca_object)
        probability = probability[0,predict_encoded] * 100
        probability = str(probability[0])
        probability = probability[:4]
        graph_pca.add_trace(go.Scatter(x=[pca_object[0,0]], y=[pca_object[0,1]], mode="markers", marker_color="blueviolet", name=f"new object({predict})"))
    
    return graph_pca, predict, probability, predict_color, probability_color


if __name__ == "__main__":
    app.run_server(debug=False)